In [34]:
import pandas as pd
from tqdm import tqdm
import pickle

In [35]:
old_trainset = pickle.load(open('trainset_w.pkl', 'rb'))
old_validset = pickle.load(open('validset_w.pkl', 'rb'))
old_testset = pickle.load(open('testset_w.pkl', 'rb'))

trainset_idx = [data[0] for data in old_trainset]
validset_idx = [data[0] for data in old_validset]
testset_idx = [data[0] for data in old_testset]

In [ ]:
for i in tqdm(range(1, 910)):
    idx = f"{i:0>3}"
    data_frame = pd.read_csv(f'POP909/POP909/{idx}/melody_chord_1_beat.csv', index_col=0)
    mask_set = set()
    mask_seg = []
    for i in range(len(data_frame)):
        if sum(eval(data_frame.iloc[i]["melody"])) == 0:
            mask_seg.append(i)
        else:
            if len(mask_seg) >= 4 or (mask_seg and mask_seg[0] == 0):
                mask_set.update(mask_seg)
            mask_seg = []
    mask_set.update(mask_seg)
    data_frame.drop(mask_set).to_csv(f'POP909/POP909/{idx}/melody_chord_1_beat_clean.csv')

100%|██████████| 909/909 [00:09<00:00, 98.08it/s] 


In [49]:
def get_repr(df):
    chords = []
    melody_reprs = []
    indices = []
    for i in range(len(df)):
        chord_str = df.iloc[i]["chord"][1:-1]
        chord_repr = [int(float(x)) for x in chord_str.split()]
        chords.append(chord_repr)
        melody_reprs.append(eval(df.iloc[i]["melody"]))
    transition_progress = [0.] * (len(chords) + 1)
    next_chords = [[0] * 12] + [None] * len(chords)
    last_chord_repr = [0] * 12
    start_idx = 0
    last_length = 1
    for i, chord_repr in enumerate(chords):
        if chord_repr == last_chord_repr:
            last_length += 1
        else:
            end_idx = start_idx + last_length
            for j in range(start_idx, end_idx):
                transition_progress[j] = (end_idx - j - 1) / last_length
                next_chords[j] = chord_repr
            start_idx = end_idx
            last_length = 1
            last_chord_repr = chord_repr
    end_idx = start_idx + last_length
    for j in range(start_idx, end_idx):
        transition_progress[j] = (end_idx - j - 1) / last_length
        next_chords[j] = [0] * 12
    return transition_progress, next_chords, melody_reprs, chords


In [50]:
new_trainset = []
new_validset = []
new_testset = []

for i in tqdm(range(1, 910)):
    idx = f"{i:0>3}"
    data_frame = pd.read_csv(f'POP909/POP909/{idx}/melody_chord_1_beat_clean.csv', index_col=0)
    transition_progress, next_chords, melody_reprs, chords = get_repr(data_frame)
    if idx in trainset_idx:
        new_trainset.append((idx, transition_progress, next_chords, melody_reprs, chords))
    elif idx in validset_idx:
        new_validset.append((idx, transition_progress, next_chords, melody_reprs, chords))
    else:
        new_testset.append((idx, transition_progress, next_chords, melody_reprs, chords))
pickle.dump(new_trainset, open(f'new_trainset.pkl', 'wb'))
pickle.dump(new_validset, open(f'new_validset.pkl', 'wb'))
pickle.dump(new_testset, open(f'new_testset.pkl', 'wb'))

100%|██████████| 909/909 [00:09<00:00, 97.62it/s] 


In [8]:
len(new_trainset), len(new_validset), len(new_testset)

(709, 100, 100)

In [58]:
def get_repr2(df):
    chords = []
    melody_reprs = []
    for i in range(len(df)):
        chord_str = df.iloc[i]["chord"][1:-1]
        chord_repr = [int(float(x)) for x in chord_str.split()]
        chords.append(chord_repr)
        melody_reprs.append(eval(df.iloc[i]["melody"]))
    reprs = [[0] * 13]
    last_chord_repr = None
    last_length = -1
    for i, chord_repr in enumerate(chords):
        last_length += 1
        if chord_repr != last_chord_repr or i == len(chords) - 1 or last_length >= 8:
            if last_chord_repr is not None:
                reprs.append(last_chord_repr + [last_length])
                last_length = 0
            last_chord_repr = chord_repr
    return melody_reprs, reprs

In [59]:
new_trainset = []
new_validset = []
new_testset = []

for i in tqdm(range(1, 910)):
    idx = f"{i:0>3}"
    data_frame = pd.read_csv(f'POP909/POP909/{idx}/melody_chord_1_beat.csv', index_col=0)
    melody_reprs, chord_reprs = get_repr2(data_frame)
    if idx in trainset_idx:
        new_trainset.append((idx, melody_reprs, chord_reprs))
    elif idx in validset_idx:
        new_validset.append((idx, melody_reprs, chord_reprs))
    else:
        new_testset.append((idx, melody_reprs, chord_reprs))
pickle.dump(new_trainset, open(f'new_trainset2.pkl', 'wb'))
pickle.dump(new_validset, open(f'new_validset2.pkl', 'wb'))
pickle.dump(new_testset, open(f'new_testset2.pkl', 'wb'))

100%|██████████| 909/909 [00:12<00:00, 72.10it/s]


In [30]:
idx, melody_reprs, chord_reprs = new_trainset[0]
print(len(melody_reprs), len(chord_reprs))

187 108


In [32]:
n = 0
for chord in new_trainset[0][2]:
    n += chord[-1]
n

187